In [1]:
import pandas as pd
import xmlrpc.client
import re


In [2]:
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# 1. Obtener las órdenes de producción ya cerradas (hechas por manufactura)
ordenes_produccion = models.execute_kw(
    db, uid, password,
    'mrp.production', 'search_read',
    [[('state', '=', 'done')]],   # Solo cerradas
    {'fields': ['id','name'#'date_start'
                , 'product_id', 'product_qty','valor_unitario_mrp','date_finished']}
)



ops = pd.DataFrame(ordenes_produccion) # todas las OP cerradas

#Demora 10 segundos

In [3]:
print(f"Primera fecha encontrada",ops["date_finished"].min())
print(f"Ultima fecha encontrada",ops["date_finished"].max())

print("_________________________________________________________")
print("")
ops.info()

Primera fecha encontrada 2016-05-05 12:56:41
Ultima fecha encontrada 2025-09-16 15:47:50
_________________________________________________________

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12012 entries, 0 to 12011
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   valor_unitario_mrp  12012 non-null  float64
 1   product_id          12012 non-null  object 
 2   date_finished       12012 non-null  object 
 3   product_qty         12012 non-null  float64
 4   id                  12012 non-null  int64  
 5   name                12012 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 563.2+ KB


In [4]:
ops["date_finished"] = pd.to_datetime(ops["date_finished"])

# 1) Quedarnos con el nombre del producto dentro de product_id
#    (Odoo devuelve [id, "nombre"])
ops['product_name_raw'] = ops['product_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

# 2) Quitar el código entre corchetes "[...]" + espacios iniciales
# 3) Quitar todo lo que esté dentro de paréntesis "(...)" + espacios
ops['producto'] = (
    ops['product_name_raw']
      .astype(str)
      .str.replace(r'^\s*\[.*?\]\s*', '', regex=True)   # elimina "[CODE] "
      .str.replace(r'\(.*?\)', '', regex=True)          # elimina "(...)"
      .str.strip()
)

# (opcional) normalizar espacios internos
ops['producto'] = ops['producto'].str.replace(r'\s+', ' ', regex=True)

ops['product_ref'] = ops["product_name_raw"].str.extract(r"\[([A-Z0-9]+)\]")


ops.head()

,valor_unitario_mrp,product_id,date_finished,product_qty,id,name,product_name_raw,producto,product_ref
0,7481.39,"[17320, [DAB02995025] DA2995 FILTRO AIRE TOYOT...",2018-03-28 13:00:43,1001.0,2388,O02445,"[DAB02995025] DA2995 FILTRO AIRE TOYOTA, MAZDA...","DA2995 FILTRO AIRE TOYOTA, MAZDA,FORD",DAB02995025
1,104539.15,"[50343, [DAB08276025] DA8276 FILTRO AIRE - FOT...",2025-09-11 16:28:42,1.0,13402,MO13131,[DAB08276025] DA8276 FILTRO AIRE - FOTON (025 ...,DA8276 FILTRO AIRE - FOTON,DAB08276025
2,15743.62,"[25666, [DAB08170025] DA8170 FILTRO AIRE CAMIO...",2025-09-12 14:10:02,300.0,13396,MO13125,[DAB08170025] DA8170 FILTRO AIRE CAMIONES JMC ...,DA8170 FILTRO AIRE CAMIONES JMC CARRYING PLUS ...,DAB08170025
3,11909.12,"[17684, [DAB04783025] DA4783 FILTRO AIRE SEGUR...",2025-09-12 14:08:39,400.0,13395,MO13124,[DAB04783025] DA4783 FILTRO AIRE SEGURIDAD- C...,"DA4783 FILTRO AIRE SEGURIDAD- CHEV.FFR,FTR, HI...",DAB04783025
4,12435.46,"[17702, [DAB04864025] DA4864 FILTRO AIRE 2_ CA...",2025-09-12 14:07:19,100.0,13392,MO13121,[DAB04864025] DA4864 FILTRO AIRE 2_ CATERPILLA...,"DA4864 FILTRO AIRE 2_ CATERPILLAR, JOHN DEERE.",DAB04864025


In [5]:
ops_def = ops[["name","product_ref","producto","valor_unitario_mrp","product_qty","date_finished"]]
ops_def.head()


,name,product_ref,producto,valor_unitario_mrp,product_qty,date_finished
0,O02445,DAB02995025,"DA2995 FILTRO AIRE TOYOTA, MAZDA,FORD",7481.39,1001.0,2018-03-28 13:00:43
1,MO13131,DAB08276025,DA8276 FILTRO AIRE - FOTON,104539.15,1.0,2025-09-11 16:28:42
2,MO13125,DAB08170025,DA8170 FILTRO AIRE CAMIONES JMC CARRYING PLUS ...,15743.62,300.0,2025-09-12 14:10:02
3,MO13124,DAB04783025,"DA4783 FILTRO AIRE SEGURIDAD- CHEV.FFR,FTR, HI...",11909.12,400.0,2025-09-12 14:08:39
4,MO13121,DAB04864025,"DA4864 FILTRO AIRE 2_ CATERPILLAR, JOHN DEERE.",12435.46,100.0,2025-09-12 14:07:19


In [6]:
import pandas as pd
import numpy as np

# Asegúrate que las fechas estén en datetime
ops_def["date_finished"] = pd.to_datetime(ops_def["date_finished"])

# Ordenar por producto y fecha
ops_sorted = ops_def.sort_values(["product_ref", "date_finished"])

# 1️⃣ Asegúrate de que esté ordenado
ops_sorted = ops_def.sort_values(["product_ref", "date_finished"])

# 2️⃣ Crear las columnas del 1 anterior
ops_sorted["costo_anterior"] = ops_sorted.groupby("product_ref")["valor_unitario_mrp"].shift(1)
ops_sorted["op_anterior"]    = ops_sorted.groupby("product_ref")["name"].shift(1)
ops_sorted["qty_anterior"]   = ops_sorted.groupby("product_ref")["product_qty"].shift(1)
ops_sorted["fecha_anterior"] = ops_sorted.groupby("product_ref")["date_finished"].shift(1)

# 3️⃣ Crear las columnas del 2 anteriores
ops_sorted["costo_anterior_2"] = ops_sorted.groupby("product_ref")["valor_unitario_mrp"].shift(2)
ops_sorted["op_anterior_2"]    = ops_sorted.groupby("product_ref")["name"].shift(2)
ops_sorted["qty_anterior_2"]   = ops_sorted.groupby("product_ref")["product_qty"].shift(2)
ops_sorted["fecha_anterior_2"] = ops_sorted.groupby("product_ref")["date_finished"].shift(2)

# 4️⃣ Variaciones con el 1 anterior (manteniendo los nombres originales)
ops_sorted["variacion"] = ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior"]
ops_sorted["variacion_abs"] = np.abs(ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior"])
ops_sorted["variacion_pct"] = np.where(
    ops_sorted["costo_anterior"] == 0,
    np.nan,
    (ops_sorted["variacion_abs"] / ops_sorted["costo_anterior"])
)

# 5️⃣ Variaciones con el 2 anterior (con sufijo _2)
ops_sorted["variacion_2"] = ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior_2"]
ops_sorted["variacion_abs_2"] = np.abs(ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior_2"])
ops_sorted["variacion_pct_2"] = np.where(
    ops_sorted["costo_anterior_2"] == 0,
    np.nan,
    (ops_sorted["variacion_abs_2"] / ops_sorted["costo_anterior_2"])
)


# impacto firmado (puede ser positivo o negativo)
ops_sorted["impacto_signed"] = (ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior"]) * ops_sorted["product_qty"]
# impacto firmado (puede ser positivo o negativo)
ops_sorted["impacto_signed_2"] = (ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior_2"]) * ops_sorted["product_qty"]



# impacto absoluto (valor absoluto en pesos, usando qty actual)
ops_sorted["impacto_abs"] = ops_sorted["variacion_abs"] * ops_sorted["product_qty"]
ops_sorted["impacto_abs_2"] = ops_sorted["variacion_abs_2"] * ops_sorted["product_qty"]






def flag_variacion(pct, abs_val):
    if pd.isna(pct):
        return "Sin histórico"
    if pct > 20 or abs_val > 5000000:   # ejemplo: 20% o más de $5MM
        return "Rojo"
    if pct > 5 or abs_val > 1000000:    # ejemplo: entre 5% y 20% o más de $1MM
        return "Amarillo"
    return "Verde"

ops_sorted["flag_var1"] = ops_sorted.apply(lambda r: flag_variacion(r["variacion_pct"], r["impacto_abs"]), axis=1)
ops_sorted["flag_var2"] = ops_sorted.apply(lambda r: flag_variacion(r["variacion_pct_2"], r["impacto_abs_2"]), axis=1)






# Exportar a Excel
ops_sorted.to_excel("historial_costos_con_variacion.xlsx", index=False)

print("✅ Archivo generado: historial_costos_con_variacion.xlsx")


/tmp/ipykernel_146684/3997407613.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ops_def["date_finished"] = pd.to_datetime(ops_def["date_finished"])


✅ Archivo generado: historial_costos_con_variacion.xlsx


In [7]:
ops_sorted.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12012 entries, 10076 to 774
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   name                12012 non-null  object        
 1   product_ref         12000 non-null  object        
 2   producto            12012 non-null  object        
 3   valor_unitario_mrp  12012 non-null  float64       
 4   product_qty         12012 non-null  float64       
 5   date_finished       12012 non-null  datetime64[ns]
 6   costo_anterior      11253 non-null  float64       
 7   op_anterior         11253 non-null  object        
 8   qty_anterior        11253 non-null  float64       
 9   fecha_anterior      11253 non-null  datetime64[ns]
 10  costo_anterior_2    10584 non-null  float64       
 11  op_anterior_2       10584 non-null  object        
 12  qty_anterior_2      10584 non-null  float64       
 13  fecha_anterior_2    10584 non-null  datetime64[ns